# BYOM Image Classifier Xplainer

ChestDetect X-AI is an advanced deep learning solution specifically designed for chest X-ray diagnosis. The solution utilizes SOTA DL model to detect and diagnose the medical conditions such as COVID-19, Lung opacity, and Viral Pneumonia. It generates activation map and highlights the respective X-ray images that explains the presence of any of the three medical conditions. 

### Prerequisite

To run this algorithm you need to have access to the following AWS Services:
- Access to AWS SageMaker and the model package.
- An S3 bucket to specify input/output.
- Role for AWS SageMaker to access input/output from S3.

This sample notebook shows you how to deploy medical xai using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

#### Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to Synthetic Data Evaluation. If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

#### Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Output Result](#D.-Output-Result)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Perform batch inference](#3.-Perform-batch-inference) 
4. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

#### Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

In [ ]:
!pip install -qU sagemaker

In [ ]:
model_package_arn='byom-image-xplainer'

In [ ]:
import os
from PIL import Image

import json
import boto3
from zipfile import ZipFile
import uuid

import sagemaker as sage
from sagemaker import ModelPackage
from sagemaker import get_execution_role

role = get_execution_role()

sagemaker_session = sage.Session()

bucket = sagemaker_session.default_bucket()
bucket

In [ ]:
artifacts_dir = 
content_type = "application/zip"

batch_transform_inference_instance_type = "ml.m5.xlarge"

In [ ]:
def predict_wrapper(endpoint, session):
    return sage.predictor.Predictor(endpoint, session, content_type)

In [ ]:
model = ModelPackage(role=role,
                    model_package_arn=model_package_arn, 
                    sagemaker_session=sagemaker_session,
                    predictor_cls=predict_wrapper)

### 2. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see these links:
1. [How it works](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html)
2. [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

In [ ]:
bucket_folder = "byom-image-xplainer-demo-test"
transform_input_folder = 'batch-inference-input/Inputs.zip'
transform_input = sagemaker_session.upload_data(transform_input_folder, key_prefix=bucket_folder) 
print("Transform input uploaded to " + transform_input)

In [ ]:
#Run the batch inference job
transformer = model.transformer(1, batch_transform_inference_instance_type, max_payload=60)
transformer.transform(transform_input, content_type=content_type)
transformer.wait()

In [ ]:
#Ouput is available on the following path
transformer.output_path

In [ ]:
s3_conn = boto3.client("s3")
bucket_name = "sagemaker-us-east-2-786796469737"

with open("batch-inference-output/response.zip", "wb") as f:
    s3_conn.download_fileobj(bucket_name, os.path.basename(transformer.output_path)+"/Inputs.zip.out", f)
    print("Output file loaded from bucket")

In [ ]:
!unzip batch-inference-output/response.zip

### 4. Clean-up

#### A. Delete the model

In [ ]:
model.delete_model()

#### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.